# Linear Regression with SystemML and Apache Spark

Uninstall all libraries that need updating.

In [1]:
#!pip uninstall systemml -y
#!pip uninstall numpy -y

Waiting for a Spark session to start...
Spark Initialization Done! ApplicationId = app-20190902041245-0012
KERNEL_ID = 2423c1f7-9e4d-4fdd-998e-35ab200cc5ba


The latest SystemML available in mid 2019 is the 1.3.0 snapshot that also works with Python 3.6 etc.

In [2]:
#!pip install --upgrade https://github.com/niketanpansare/future_of_data/raw/master/systemml-1.3.0-SNAPSHOT-python.tar.gz
!pip install numpy==1.17.1 

  Using cached https://files.pythonhosted.org/packages/75/92/57179ed45307ec6179e344231c47da7f3f3da9e2eee5c8ab506bd279ce4e/numpy-1.17.1-cp36-cp36m-manylinux1_x86_64.whl
Error checking for conflicts.
Traceback (most recent call last):
  File "/opt/ibm/conda/miniconda36/lib/python3.6/site-packages/pip/_vendor/pkg_resources/__init__.py", line 2584, in version
    return self._version
  File "/opt/ibm/conda/miniconda36/lib/python3.6/site-packages/pip/_vendor/pkg_resources/__init__.py", line 2691, in __getattr__
    raise AttributeError(attr)
AttributeError: _version

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/opt/ibm/conda/miniconda36/lib/python3.6/site-packages/pip/_internal/commands/install.py", line 503, in _warn_about_conflicts
    package_set, _dep_info = check_install_conflicts(to_install)
  File "/opt/ibm/conda/miniconda36/lib/python3.6/site-packages/pip/_internal/operations/check.py", line 108, in check_install_co

In [3]:
!ln -s -f /home/spark/shared/user-libs/python3/systemml/systemml-java/systemml-1.3.0-SNAPSHOT-extra.jar ~/user-libs/spark2/systemml-1.3.0-SNAPSHOT-extra.jar
!ln -s -f /home/spark/shared/user-libs/python3/systemml/systemml-java/systemml-1.3.0-SNAPSHOT.jar ~/user-libs/spark2/systemml-1.3.0-SNAPSHOT.jar

In [4]:
from systemml import MLContext

ml = MLContext(spark)
ml.version()

'1.3.0-SNAPSHOT'

In [5]:
# And print the information of SystemML version
print(ml.info())

Archiver-Version: Plexus Archiver
Artifact-Id: systemml
Build-Jdk: 1.8.0_181
Build-Time: 2019-03-19 23:17:12 UTC
Built-By: npansar
Created-By: Apache Maven 3.5.4
Group-Id: org.apache.systemml
Main-Class: org.apache.sysml.api.DMLScript
Manifest-Version: 1.0
Minimum-Recommended-Spark-Version: 2.3.0
Version: 1.3.0-SNAPSHOT



In [6]:
# The code was removed by Watson Studio for sharing.

In [7]:
df = pd.read_csv(streaming_body_1)
df.head()

,Unnamed: 0,area,peri,shape,perm
0,1,4990,2791.90,0.090330,6.3
1,2,7002,3892.60,0.148622,6.3
2,3,7558,3930.66,0.183312,6.3
3,4,7352,3869.32,0.117063,6.3
4,5,7943,3948.54,0.122417,17.1


In [8]:
df = pd.DataFrame(df, columns=['area','peri','shape','perm'])
df.head()

,area,peri,shape,perm
0,4990,2791.90,0.090330,6.3
1,7002,3892.60,0.148622,6.3
2,7558,3930.66,0.183312,6.3
3,7352,3869.32,0.117063,6.3
4,7943,3948.54,0.122417,17.1


In [9]:
df_y = df.drop(['area','peri','shape'], axis=1)
df_y.head()

,perm
0,6.3
1,6.3
2,6.3
3,6.3
4,17.1


In [10]:
import numpy as np

y = np.array(df_y)
y[0:5]

array([[ 6.3],
       [ 6.3],
       [ 6.3],
       [ 6.3],
       [17.1]])

In [11]:
df_X = df.drop(['perm'], axis=1)
df_X.head()

,area,peri,shape
0,4990,2791.90,0.090330
1,7002,3892.60,0.148622
2,7558,3930.66,0.183312
3,7352,3869.32,0.117063
4,7943,3948.54,0.122417


In [12]:
X = np.array(df_X)
X[0:5]

array([[4.99000e+03, 2.79190e+03, 9.03296e-02],
       [7.00200e+03, 3.89260e+03, 1.48622e-01],
       [7.55800e+03, 3.93066e+03, 1.83312e-01],
       [7.35200e+03, 3.86932e+03, 1.17063e-01],
       [7.94300e+03, 3.94854e+03, 1.22417e-01]])

SystemML should accept data as numpy array and also as pandas dataframe. I will feed it as array!

In [13]:
from sklearn.utils import shuffle

seed = 0
np.random.seed(seed)

X = shuffle(X, random_state=seed)
y = shuffle(y, random_state=seed)
print(X[0:5])
print(y[0:5])

[[4.89500e+03 1.23966e+03 3.41273e-01]
 [7.94300e+03 3.94854e+03 1.22417e-01]
 [7.41600e+03 1.91624e+03 1.91802e-01]
 [6.77500e+03 1.72814e+03 3.11646e-01]
 [5.98000e+03 1.42676e+03 1.97653e-01]]
[[890. ]
 [ 17.1]
 [740. ]
 [890. ]
 [950. ]]


At this point you would split the model into train and test set with sklearn. But the data set is too small for this.

In [14]:
#from systemml.mllearn import LogisticRegression
from systemml.mllearn import LinearRegression
from pyspark.sql import SparkSession

sparkSession = SparkSession.builder.getOrCreate()
logistic = LinearRegression(sparkSession)

In [15]:
model = logistic.fit(X, y)

BEGIN LINEAR REGRESSION SCRIPT
Reading X and Y...
Running the CG algorithm...
||r|| initial value = 1.2550571663279374E8,  target value = 125.50571663279375
Iteration 1:  ||r|| / ||r init|| = 0.11135166118709607
Iteration 2:  ||r|| / ||r init|| = 3.200522777969612E-5
Iteration 3:  ||r|| / ||r init|| = 1.8851794008794281E-6
Iteration 4:  ||r|| / ||r init|| = 1.7816699070061662E-6
Iteration 5:  ||r|| / ||r init|| = 0.007085696187536821
Iteration 6:  ||r|| / ||r init|| = 8.747532691590887E-11
The CG algorithm is done.
Computing the statistics...
AVG_TOT_Y,415.45
STDEV_TOT_Y,437.8182261497757
AVG_RES_Y,-2.6060448613899705E-8
STDEV_RES_Y,246.01471748689235
DISPERSION,60523.241220155454
R2,0.7044102825997469
ADJUSTED_R2,0.6842564382315479
R2_NOBIAS,0.7044102825997469
ADJUSTED_R2_NOBIAS,0.6842564382315479
Writing the output matrix...
END LINEAR REGRESSION SCRIPT
SystemML Statistics:
Total execution time:		0.061 sec.
Number of executed Spark inst:	2.




In [16]:
print('LogisticRegression score: {}'.format(model.score(X, y)))

SystemML Statistics:
Total execution time:		0.003 sec.
Number of executed Spark inst:	1.


LogisticRegression score: 0.7044102825997469


In [17]:
y_hat = model.predict(X)

SystemML Statistics:
Total execution time:		0.002 sec.
Number of executed Spark inst:	1.




In [18]:
predictions = np.column_stack((y, y_hat));
print(predictions[:10])

[[ 890.          813.0513966 ]
 [  17.1         -37.25128228]
 [ 740.          676.15893185]
 [ 890.          790.07305719]
 [ 950.          718.65722751]
 [ 100.          850.26196315]
 [ 950.          834.69195848]
 [1300.          918.22800598]
 [  17.1        -111.76230345]
 [ 119.          -64.67073582]]
